In [255]:
import numpy as np
from scipy.optimize import brentq
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u
from scipy.integrate import dblquad


In [256]:
print(const.M_sun)
print(const.G)
print(const.R_jup)
print(const.a)

  Name   = Solar mass
  Value  = 1.988409870698051e+30
  Uncertainty  = 4.468805426856864e+25
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018
  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018
  Name   = Nominal Jupiter equatorial radius
  Value  = 71492000.0
  Uncertainty  = 0.0
  Unit  = m
  Reference = IAU 2015 Resolution B 3


AttributeError: module 'astropy.constants' has no attribute 'a'

In [ ]:

G = const.G.value
mA = const.M_sun.value
muA = G * mA
mB =  const.M_jup.value  # Mass of Jupiter
muB = G * mB
M_PBH = 1e-13 * const.M_sun.value
rAB = (5.2*u.au).to(u.m).value
vB_mag = 13.06 * 1e3  # km/s to m/s
kappa = 25
epsilon = 0.1
v_inf = 20.23 * 1e3  # km/s to m/s
r_jupiter = const.R_jup.to(u.m).value
A_jupiter = np.pi * r_jupiter**2

def sigma_cap(v1, v1_prime, vB, v_esc, muB):
    """
    Capture cross-section σ_cap(v1).

    Parameters
    ----------
    v1 : float
        Incoming velocity.
    v1_prime : float
        Incoming velocity in Body B frame.
    vB : float
        Orbital velocity of body B.
    vesc : float
        Escape velocity of body A or system.
    muB : float
        Reduced mass associated with body B (μ_B).

    Returns
    -------
    float
        σ_cap(v1)
    """
    numerator = np.pi * muB**2 * ((v1_prime**2 - vB**2)**2 - v_esc**4)
    denominator = (v1**2 - v_esc**2)**2 * v1_prime**4
    return numerator / denominator

def v_esc(muA, rAB):
    """
    Escape velocity v_esc.

    Parameters
    ----------
    mA : float
        Mass of body A.
    rAB : float
        Separation between bodies A and B.

    Returns
    -------
    float
        v_esc
    """

    return np.sqrt(2 * muA / rAB)

def r_close(epsilon, mA, mB, rAB):
    """
    Closest approach distance r_close.

    Parameters
    ----------
    epsilon : float
        Energy dissipation parameter.
    mA : float
        Mass of body A.
    mB : float
        Mass of body B.
    rAB : float
        Separation between bodies A and B.

    Returns
    -------
    float
        r_close
    """

    return rAB * (epsilon * mB / mA)**(1/3)

def v_1(v_inf, muA, muB, rAB, r_close):
    """
    Incoming velocity v1.

    Parameters
    ----------
    v_inf : float
        Velocity at infinity.
    mA : float
        Mass of body A.
    rAB : float
        Separation between bodies A and B.


    Returns
    -------
    float
        v1
    """

    term1 = v_inf**2 + (2 * muA / rAB)
    term2 = (2 * muB / r_close)
    return np.sqrt(term1 + term2)

def v_1_prime(v1_mag, vB_mag, lambda_1, beta_1):
    """
    Incoming velocity in Body B frame v1'.
    Parameters
    ----------
    v1_mag : float
        Incoming velocity magnitude.
    vB_mag : float
        Orbital velocity magnitude of body B.
    lambda_1 : float
        Scattering angle.
    beta_1 : float
        Impact parameter angle. 
    Returns
    -------
    float
        v1'
    """
    # B moves along +y
    vB_vec = np.array([0.0, vB_mag, 0.0])
    # v1 direction set by (β1, λ1) as in Lehmann fig. 2
    v1_vec = v1_mag * np.array([
        np.sin(lambda_1) * np.cos(beta_1),
        np.cos(lambda_1) * np.cos(beta_1),
        -np.sin(beta_1),
    ])
    v1_prime_vec = v1_vec - vB_vec
    return np.linalg.norm(v1_prime_vec)

def chi(muB,v1_mag, vB_mag, vesc, sigma_cap, lambda_1, beta_1, rclose):
    """
    Indicator function to check kinematical allowance χ.

    Parameters
    ----------
    v_1 : float
        Incoming velocity.
    vB : float
        Orbital velocity of body B.
    lambda_1 : float
        Scattering angle.

    Returns
    -------
    boolean
        χ
    """

    # Condition 1
    v_max = vesc + 2*vB_mag
    condition1 = v1_mag < v_max

    # Condition 2
    condition2 = sigma_cap > 0


    # Condition 3 (not needed as sigma_cap already incorporates this)

    # B moves along +y
    vB_vec = np.array([0.0, vB_mag, 0.0])
    # v1 direction set by (β1, λ1) as in Lehmann fig. 2
    v1_vec = v1_mag * np.array([
        np.sin(lambda_1) * np.cos(beta_1),
        np.cos(lambda_1) * np.cos(beta_1),
        -np.sin(beta_1),
    ])

    v1_prime_vec = v1_vec - vB_vec
    v1_prime_mag = np.linalg.norm(v1_prime_vec)
    v1_prime_vec_xy = v1_prime_vec.copy()
    v1_prime_vec_xy[2] = 0.0
    v1_prime_mag_xy = np.linalg.norm(v1_prime_vec_xy)
    g1 = 4*muB*(np.dot(vB_vec, v1_prime_vec))*(v1_prime_mag**6)*v1_prime_vec[2]
    g2 = 4*(muB**3)*(np.dot(vB_vec, v1_prime_vec))*(v1_prime_mag**2)*v1_prime_vec[2]
    g3 = 4*muB*(v1_prime_mag**7)*(np.cross(v1_prime_vec, vB_vec))[2]
    g4 = np.sign(v1_prime_vec[1])*(v1_prime_mag**8)*v1_prime_mag_xy*(v1_mag**2 - vesc**2)
    g5 = 2 * np.sign(v1_prime_vec[1]) * (muB**2) * (v1_prime_mag**4) * v1_prime_mag_xy * (v1_mag**2 + vB_mag**2 - vesc**2)
    R = np.sqrt(g2/g1 + (g1**2 + g3**2)/(4*g4**2) - g5/g4)
    # guard against singular geometries
    if g1 == 0 or g4 == 0:
        return 0.0

    expr = g2/g1 + (g1**2 + g3**2)/(4*g4**2) - g5/g4
    if expr <= 0:
        return 0.0
    
    condition3 = R < rclose

    # Condition 4 impact parameter should be larger than Jupiter radius
    impact_parameter = -1/(2*g4) * (g3 + g1)
    r_jupiter = const.R_jup.to(u.m).value
    condition4 = impact_parameter > r_jupiter
    

    result = 1.0 if (condition1 and condition2 and condition3 and condition4) else 0.0

    return result

def dsigma(lambda_1, cosbeta_1, sigma_cap, chi):
    """
    Differential cross-section dσ/dΩ.

    Parameters
    ----------
    lambda_1 : float
        Scattering angle.
    beta_1 : float
        Impact parameter.
    sigma_cap : float
        Capture cross-section.
    chi : float
        Kinematical allowance.

    Returns
    -------
    float
        dσ/dΩ
    """

    return sigma_cap * chi / (2*np.pi)

In [258]:
vesc = v_esc(muA, rAB)
rclose = r_close(epsilon, mA, mB, rAB)
v1_mag = v_1(v_inf, muA, muB, rAB, rclose)

def integrand(cosbeta_1, lambda_1):  # func(y, x): y = cosbeta_1, x = lambda_1
    beta_1 = np.arccos(cosbeta_1)
    v1p = v_1_prime(v1_mag, vB_mag, lambda_1, beta_1)
    sigcap = sigma_cap(v1_mag, v1p, vB_mag, vesc, muB)
    allow = chi(muB,v1_mag, vB_mag, vesc, sigcap, lambda_1, beta_1, rclose)
    return dsigma(lambda_1, cosbeta_1, sigcap, allow)

e = 0.1
res, err = dblquad(
    integrand,
    0, np.pi,
    lambda cosbeta_1: -1,
    lambda cosbeta_1: 1
)
print("Integrated cross section:", res, " +/- ", err)

/tmp/ipykernel_217702/2998484907.py:184: RuntimeWarning: invalid value encountered in sqrt
  R = np.sqrt(g2/g1 + (g1**2 + g3**2)/(4*g4**2) - g5/g4)
/grad/a.saricaoglu/.conda/envs/compas/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1260: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/grad/a.saricaoglu/.conda/envs/compas/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1260: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It

Integrated cross section: 9.0357093545181e+16  +/-  10289457179344.053


In [259]:
print(((res)/A_jupiter))

5.62726654507256


without chi:
14.97410460456744 for 0, 2pi for lambda & 0, pi for beta
7.4870481200022825 for 0, pi for lambda & 0, pi for beta
with chi:
14.999128490217245 for 0, 2pi for lambda & 0, pi for beta
7.499568974815175 for 0, pi for lambda & 0, pi for beta
with chi & b check:
11.254511033661727 for 0, 2pi for lambda & 0, pi for beta
5.62726654507256 for 0, pi for lambda & 0, pi for beta


In [ ]:

G = const.G.value
mA = const.M_sun.value
muA = G * mA
mB =  const.M_jup.value  # Mass of Jupiter
muB = G * mB
M_PBH = 1e-13 * const.M_sun.value
rAB = (5.2*u.au).to(u.m).value
vB_mag = 13.06 * 1e3  # km/s to m/s
kappa = 25
epsilon = 0.1
v_inf = 20.23 * 1e3  # km/s to m/s
r_jupiter = const.R_jup.to(u.m).value
A_jupiter = np.pi * r_jupiter**2

def sigma_cap(v1, v1_prime, vB, v_esc, muB):
    """
    Capture cross-section σ_cap(v1).

    Parameters
    ----------
    v1 : float
        Incoming velocity.
    v1_prime : float
        Incoming velocity in Body B frame.
    vB : float
        Orbital velocity of body B.
    vesc : float
        Escape velocity of body A or system.
    muB : float
        Reduced mass associated with body B (μ_B).

    Returns
    -------
    float
        σ_cap(v1)
    """
    numerator = np.pi * muB**2 * ((v1_prime**2 - vB**2)**2 - v_esc**4)
    denominator = (v1**2 - v_esc**2)**2 * v1_prime**4
    return numerator / denominator

def v_esc(muA, rAB):
    """
    Escape velocity v_esc.

    Parameters
    ----------
    mA : float
        Mass of body A.
    rAB : float
        Separation between bodies A and B.

    Returns
    -------
    float
        v_esc
    """

    return np.sqrt(2 * muA / rAB)

def r_close(epsilon, mA, mB, rAB):
    """
    Closest approach distance r_close.

    Parameters
    ----------
    epsilon : float
        Energy dissipation parameter.
    mA : float
        Mass of body A.
    mB : float
        Mass of body B.
    rAB : float
        Separation between bodies A and B.

    Returns
    -------
    float
        r_close
    """

    return rAB * (epsilon * mB / mA)**(1/3)

def v_1(v_inf, muA, muB, rAB, r_close):
    """
    Incoming velocity v1.

    Parameters
    ----------
    v_inf : float
        Velocity at infinity.
    mA : float
        Mass of body A.
    rAB : float
        Separation between bodies A and B.


    Returns
    -------
    float
        v1
    """

    term1 = v_inf**2 + (2 * muA / rAB)
    term2 = (2 * muB / r_close)
    return np.sqrt(term1 + term2)

def v_1_prime(v1_mag, vB_mag, lambda_1, beta_1):
    """
    Incoming velocity in Body B frame v1'.
    Parameters
    ----------
    v1_mag : float
        Incoming velocity magnitude.
    vB_mag : float
        Orbital velocity magnitude of body B.
    lambda_1 : float
        Scattering angle.
    beta_1 : float
        Impact parameter angle. 
    Returns
    -------
    float
        v1'
    """
    # B moves along +y
    vB_vec = np.array([0.0, vB_mag, 0.0])
    # v1 direction set by (β1, λ1) as in Lehmann fig. 2
    v1_vec = v1_mag * np.array([
        np.cos(lambda_1) * np.cos(beta_1),
        np.sin(lambda_1) * np.cos(beta_1),
        np.sin(beta_1),
    ])
    v1_prime_vec = v1_vec - vB_vec
    return np.linalg.norm(v1_prime_vec)

def chi(muB,v1_mag, vB_mag, vesc, sigma_cap, lambda_1, beta_1, rclose):
    """
    Indicator function to check kinematical allowance χ.

    Parameters
    ----------
    v_1 : float
        Incoming velocity.
    vB : float
        Orbital velocity of body B.
    lambda_1 : float
        Scattering angle.

    Returns
    -------
    boolean
        χ
    """

    # Condition 1
    v_max = vesc + 2*vB_mag
    condition1 = v1_mag < v_max

    # Condition 2
    condition2 = sigma_cap > 0


    # Condition 3 (not needed as sigma_cap already incorporates this)

    # B moves along +y
    vB_vec = np.array([0.0, vB_mag, 0.0])
    # v1 direction set by (β1, λ1) as in Lehmann fig. 2
    v1_vec = v1_mag * np.array([
        np.cos(lambda_1) * np.cos(beta_1),
        np.sin(lambda_1) * np.cos(beta_1),
        np.sin(beta_1),
    ])

    v1_prime_vec = v1_vec - vB_vec
    v1_prime_mag = np.linalg.norm(v1_prime_vec)
    v1_prime_vec_xy = v1_prime_vec.copy()
    v1_prime_vec_xy[2] = 0.0
    v1_prime_mag_xy = np.linalg.norm(v1_prime_vec_xy)
    g1 = 4*muB*(np.dot(vB_vec, v1_prime_vec))*(v1_prime_mag**6)*v1_prime_vec[2]
    g2 = 4*(muB**3)*(np.dot(vB_vec, v1_prime_vec))*(v1_prime_mag**2)*v1_prime_vec[2]
    g3 = 4*muB*(v1_prime_mag**7)*(np.cross(v1_prime_vec, vB_vec))[2]
    g4 = np.sign(v1_prime_vec[1])*(v1_prime_mag**8)*v1_prime_mag_xy*(v1_mag**2 - vesc**2)
    g5 = 2 * np.sign(v1_prime_vec[1]) * (muB**2) * (v1_prime_mag**4) * v1_prime_mag_xy * (v1_mag**2 + vB_mag**2 - vesc**2)
    R = np.sqrt(g2/g1 + (g1**2 + g3**2)/(4*g4**2) - g5/g4)

    condition3 = R < rclose

    result = 1.0 if (condition1 and condition2 and condition3) else 0.0

    return result

def dsigma(lambda_1, cosbeta_1, sigma_cap, chi):
    """
    Differential cross-section dσ/dΩ.

    Parameters
    ----------
    lambda_1 : float
        Scattering angle.
    beta_1 : float
        Impact parameter.
    sigma_cap : float
        Capture cross-section.
    chi : float
        Kinematical allowance.

    Returns
    -------
    float
        dσ/dΩ
    """

    return sigma_cap * chi / (2*np.pi)
vesc = v_esc(muA, rAB)
rclose = r_close(epsilon, mA, mB, rAB)
v1_mag = v_1(v_inf, muA, muB, rAB, rclose)

def integrand(cosbeta_1, lambda_1):  # func(y, x): y = cosbeta_1, x = lambda_1
    beta_1 = np.arccos(cosbeta_1)
    v1p = v_1_prime(v1_mag, vB_mag, lambda_1, beta_1)
    sigcap = sigma_cap(v1_mag, v1p, vB_mag, vesc, muB)
    allow = chi(muB,v1_mag, vB_mag, vesc, sigcap, lambda_1, beta_1, rclose)
    return dsigma(lambda_1, cosbeta_1, sigcap, allow)

e = 0.1
res, err = dblquad(
    integrand,
    0, 2*np.pi,
    lambda lam: -1,
    lambda lam: 1
)
print("Integrated cross section:", res, " +/- ", err)